# Tourism Data Processing
DS Senior Project - Noah B Johnson - Luther College

In [1]:
import pandas as pd
import numpy as np
import pycountry
import re

pd.set_option('display.float_format', lambda x: '%.3f' % x)

## World Development Indicators

In [2]:
wdi = pd.read_excel("../data/external/Tourism/WDI/Data_Extract_From_World_Development_Indicators.xlsx")

In [3]:
wdi.head(3)

,Series Name,Series Code,Country Name,Country Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,"International tourism, number of arrivals",ST.INT.ARVL,Afghanistan,AFG,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1,"International tourism, number of arrivals",ST.INT.ARVL,Albania,ALB,..,..,..,..,..,..,...,1711000,2191000,2469000,3156000,2857000,3341000,3784000,4070000,4643000,..
2,"International tourism, number of arrivals",ST.INT.ARVL,Algeria,DZA,..,..,..,..,..,..,...,1912000,2070000,2395000,2634000,2733000,2301000,1710000,2039000,2451000,..


In [4]:
wdi['Series Code'].unique()

array(['ST.INT.ARVL', 'ST.INT.XPND.MP.ZS', 'ST.INT.XPND.CD',
       'ST.INT.TRNX.CD', 'ST.INT.TVLX.CD', 'ST.INT.DPRT',
       'ST.INT.RCPT.XP.ZS', 'ST.INT.RCPT.CD', 'ST.INT.TRNR.CD',
       'ST.INT.TVLR.CD', nan], dtype=object)

In [5]:
wdi.tail(3)

,Series Name,Series Code,Country Name,Country Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
2642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2643,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2644,Last Updated: 01/30/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# nan is not a valid series code, so we should remove those rows
wdi.dropna(subset=['Series Code', 'Country Code'], inplace=True)

In [7]:
wdi.tail(3)

,Series Name,Series Code,Country Name,Country Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
2637,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,Sub-Saharan Africa (IDA & IBRD countries),TSS,..,..,..,..,..,..,...,19838943841.034,21673657225.577,23876418124.733,26142967778.798,25790138383.752,27057578741.885,24763693256.545,25325608754.148,29007199323.610,..
2638,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,Upper middle income,UMC,..,..,..,..,..,..,...,177103169507.960,193953837035.759,213249737962.092,227425606689.724,246611754348.080,246280828980.337,244234779637.110,241924733652.200,253188732331.635,..
2639,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,World,WLD,..,..,..,..,..,..,...,893141573444.509,965630203496.872,1077839164714.778,1114345084520.954,1199717631267.226,1261290553785.728,1215027166077.664,1238015449937.971,1338751045869.408,..


### Pivoting the Data
> Target format is data by year-country with indicator columns

In [8]:
# Making our keys
country_codes = wdi['Country Code'].unique()
years = wdi.columns[4:].tolist()

In [9]:
print("We will have {} rows".format(len(country_codes) * len(years)))

We will have 15576 rows


In [10]:
""" Manual transposition of data
"""

year_column = []
country_column = []
data_columns = {'ST.INT.ARVL': [],
                'ST.INT.XPND.MP.ZS': [],
                'ST.INT.XPND.CD': [],
                'ST.INT.TRNX.CD': [],
                'ST.INT.TVLX.CD': [],
                'ST.INT.DPRT': [],
                'ST.INT.RCPT.XP.ZS': [],
                'ST.INT.RCPT.CD': [],
                'ST.INT.TRNR.CD': [],
                'ST.INT.TVLR.CD': []}
for country in country_codes:
    for year in years:
        year_column.append(year)
        country_column.append(country)
        for indicator in data_columns.keys():            
            cdf = wdi.loc[(wdi['Country Code'] == country)]
            datum = cdf[cdf['Series Code'] == indicator][year].tolist()[0]
            data_columns[indicator].append(datum)

In [11]:
""" Reassign country and year to columns
"""
data_columns["Year"] = year_column
data_columns["Country Code"] = country_column

In [12]:
""" Data is correctly formatted
"""
wdi_clean = pd.DataFrame(data_columns)

### Cleaning data
> fix nans, assign correct data types, and standardize countries

In [13]:
wdi_clean.head(3)

,ST.INT.ARVL,ST.INT.XPND.MP.ZS,ST.INT.XPND.CD,ST.INT.TRNX.CD,ST.INT.TVLX.CD,ST.INT.DPRT,ST.INT.RCPT.XP.ZS,ST.INT.RCPT.CD,ST.INT.TRNR.CD,ST.INT.TVLR.CD,Year,Country Code
0,..,..,..,..,..,..,..,..,..,..,1960 [YR1960],AFG
1,..,..,..,..,..,..,..,..,..,..,1961 [YR1961],AFG
2,..,..,..,..,..,..,..,..,..,..,1962 [YR1962],AFG


In [14]:
""" Replace '..' with np.nan for better parsing
"""
wdi_clean = wdi_clean.replace('..', np.NaN)

In [15]:
wdi_clean.head(3)

,ST.INT.ARVL,ST.INT.XPND.MP.ZS,ST.INT.XPND.CD,ST.INT.TRNX.CD,ST.INT.TVLX.CD,ST.INT.DPRT,ST.INT.RCPT.XP.ZS,ST.INT.RCPT.CD,ST.INT.TRNR.CD,ST.INT.TVLR.CD,Year,Country Code
0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1960 [YR1960],AFG
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1961 [YR1961],AFG
2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1962 [YR1962],AFG


In [16]:
""" Assign all columns except year and country to float dtype
"""
wdi_clean[wdi_clean.columns[:-2]] = wdi_clean[wdi_clean.columns[:-2]].astype(float)

In [17]:
""" Check the dtypes of the dataframe
"""
wdi_clean.get_dtype_counts()

float64    10
object      2
dtype: int64

In [18]:
""" Remove the bracketed text and parse year into an integer so: '1960 [YR1960]' → int(1960)
"""
wdi_clean["Year"] = wdi_clean["Year"].apply(lambda x: re.match(r'^\d*',x)[0]).astype(int)

In [19]:
""" Check year values
"""
wdi_clean.sample(3)

,ST.INT.ARVL,ST.INT.XPND.MP.ZS,ST.INT.XPND.CD,ST.INT.TRNX.CD,ST.INT.TVLX.CD,ST.INT.DPRT,ST.INT.RCPT.XP.ZS,ST.INT.RCPT.CD,ST.INT.TRNR.CD,ST.INT.TVLR.CD,Year,Country Code
2044,nan,9.039,210000000.000,50000000.000,160000000.000,nan,4.736,114000000.000,29000000.000,85000000.000,1998,CMR
1549,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1975,BRA
12050,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1974,ARE


In [20]:
""" 
Create a mask without regions and custom groupings, 
selecting only valid country entries
"""
country_bool = []
for country in wdi_clean['Country Code'].tolist():
    try:
        pycountry.countries.lookup(country)
        country_bool.append(True)
    except LookupError:
        country_bool.append(False)

In [21]:
len(wdi_clean['Country Code'].unique())

264

In [22]:
wdi_clean = wdi_clean[country_bool]

In [23]:
len(wdi_clean['Country Code'].unique())

215

In [24]:
""" Standardize the country codes
"""
country_codes = []
for country in wdi_clean['Country Code'].tolist():
    country_codes.append(pycountry.countries.lookup(country).alpha_3)

In [25]:
wdi_clean.loc[:,'Country Code'] = country_codes

In [26]:
wdi_clean.describe()

,ST.INT.ARVL,ST.INT.XPND.MP.ZS,ST.INT.XPND.CD,ST.INT.TRNX.CD,ST.INT.TVLX.CD,ST.INT.DPRT,ST.INT.RCPT.XP.ZS,ST.INT.RCPT.CD,ST.INT.TRNR.CD,ST.INT.TVLR.CD,Year
count,4390.000,3748.000,4055.000,3289.000,3956.000,2234.000,3752.000,4266.000,3033.000,3985.000,12685.000
mean,4417208.200,6.605,4828076388.212,794672287.039,4228815999.141,8031308.268,16.952,4928297940.752,922708154.258,4476780046.173,1989.000
std,10272868.297,4.332,15399011971.733,2718791242.890,13733953517.299,16159876.350,18.848,14801029019.088,2995992210.148,12946159292.781,17.030
min,700.000,0.183,100000.000,100000.000,10000.000,1900.000,0.001,100000.000,11000.000,100000.000,1960.000
25%,199000.000,3.696,97000000.000,21000000.000,71000000.000,486250.000,4.269,119250000.000,17000000.000,107000000.000,1974.000
50%,800000.000,5.579,423000000.000,87000000.000,329367300.000,1957000.000,9.356,660000000.000,128000000.000,578000000.000,1989.000
75%,3513500.000,8.303,2628500000.000,375000000.000,2240000000.000,7331250.000,22.104,3372000000.000,593000000.000,3204000000.000,2004.000
max,86861000.000,42.441,257733000000.000,38896000000.000,257733000000.000,143035000.000,170.479,251361000000.000,44071000000.000,210748000000.000,2018.000


### Saving Data

In [27]:
""" Save the cleaned and standardized data
"""
wdi_clean.to_pickle("../data/interim/WDI.pickle")
pd.DataFrame({
    'Series Code': wdi['Series Code'].unique(),
    'Series Name': wdi['Series Name'].unique()}).to_pickle("../data/interim/WDI_DD.pickle")

## World Tourism Organization

In [30]:
uwto = pd.read_excel("../data/external/Tourism/UWTO/Arrivals of non resident tourists_visitors, depart.xls")

### Grouping by Country

In [31]:
""" 
Create a mask without regions and custom groupings, 
selecting only valid country entries
"""
country_locations = []
row = 0
for country in uwto['COUNTRY']:
    try:
        pycountry.countries.lookup(country)
        country_locations.append(row)
    except LookupError:
        pass
    row += 1
country_locations.append(len(uwto))

In [32]:
""" Create a dataframe for each country code
"""
country_slices = []
for index in range(len(country_locations)):
    try:
        country_slices.append(
            [country_locations[index], country_locations[index + 1] - 1])
    except IndexError:
        pass
nation_frames = []
for s in country_slices:
    nation_frames.append(uwto.loc[s[0]:s[1]])

In [33]:
""" Clean the dataframes and flatten into standard format
"""
clean_nations = []

for nation in nation_frames:
    nation = nation.reset_index()
    nation_name = nation['COUNTRY'][0]
    
    # list of row indexes indicating a chunk of data (inbound/outbound)
    chunk_headers = []
    row = 0
    for country in nation['COUNTRY']:
        if 'bound tourism' in country:
            chunk_headers.append(row)
        row += 1
    
    # New dfs on variable groups
    inbound = nation[chunk_headers[0]:chunk_headers[1]]
    outbound = nation[chunk_headers[1]:]   
    
    # rename variables to flatten index
    outbound.loc[1:,'COUNTRY'] = 'Outbound: ' + outbound[1:]['COUNTRY']
    inbound.loc[1:,'COUNTRY'] = 'Inbound: ' + inbound[1:]['COUNTRY']
    
    # combine variable groups and remove index rows
    rejoined = pd.concat([outbound, inbound]).dropna()
    
    # move indicators to correct column
    rejoined['Indicator'] = rejoined['COUNTRY']
    
    # set the country name to the correct value
    rejoined['COUNTRY'] = nation_name
    
    # Fix column order
    rejoined = rejoined[['COUNTRY',
                     'Indicator',
                     1995,
                     1996,
                     1997,
                     1998,
                     1999,
                     2000,
                     2001,
                     2002,
                     2003,
                     2004,
                     2005,
                     2006,
                     2007,
                     2008,
                     2009,
                     2010,
                     2011,
                     2012,
                     2013,
                     2014,
                     2015,
                     2016,
                     2017]]
    
    # add to clean list
    clean_nations.append(rejoined)
    
uwto_clean = pd.concat(clean_nations)

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Clean Values and Group Variables on Means

In [34]:
uwto_clean.head(3)

,COUNTRY,Indicator,1995,1996,1997,1998,1999,2000,2001,2002,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
7,AFGHANISTAN,Outbound: Departures - Thousands,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
8,AFGHANISTAN,Outbound: Tourism expenditure in other countri...,..,..,..,..,..,..,..,..,...,28,70,106,255,110,138,140,151,89,132
9,AFGHANISTAN,Outbound: Travel - US$ Mn,..,..,..,..,..,..,..,..,...,26,68,103,246,105,132,137,147,79,118


In [35]:
""" Replace '..' with np.nan for better parsing
"""
uwto_clean = uwto_clean.replace('..', np.NaN)

In [36]:
uwto_clean.head(3)

,COUNTRY,Indicator,1995,1996,1997,1998,1999,2000,2001,2002,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
7,AFGHANISTAN,Outbound: Departures - Thousands,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,AFGHANISTAN,Outbound: Tourism expenditure in other countri...,nan,nan,nan,nan,nan,nan,nan,nan,...,28.000,70.000,106.000,255.000,110.000,138.000,140.000,151.000,89.000,132.000
9,AFGHANISTAN,Outbound: Travel - US$ Mn,nan,nan,nan,nan,nan,nan,nan,nan,...,26.000,68.000,103.000,246.000,105.000,132.000,137.000,147.000,79.000,118.000


In [37]:
""" Assign all columns except indicator and country to float dtype
"""
uwto_clean[uwto_clean.columns[2:]] = uwto_clean[uwto_clean.columns[2:]].astype(float)

In [38]:
uwto_clean.get_dtype_counts()

float64    23
object      2
dtype: int64

In [39]:
""" Select unique
"""
uwto_clean = uwto_clean.groupby(['COUNTRY','Indicator']).max()

### Pivot the Data

In [40]:
# Move indicators to the columns index
unstacked = uwto_clean.unstack()
# Name column indexes
unstacked.columns.names = ['Year','Indicator']
# move year to row index
restacked = unstacked.stack(level=0,dropna=False)
# move country and year to columns
uwto_pivoted = restacked.reset_index()
# remove indicator label from column index
uwto_pivoted.columns.name = None

In [41]:
uwto_pivoted.head(3)

,COUNTRY,Year,Inbound: Arrivals - Thousands,Inbound: Passenger transport - US$ Mn,Inbound: Tourism expenditure in the country - US$ Mn,Inbound: Travel - US$ Mn,Outbound: Arrivals - Thousands,Outbound: Departures - Thousands,Outbound: Passenger transport - US$ Mn,Outbound: Tourism expenditure in other countries - US$ Mn,Outbound: Tourism expenditure in the country - US$ Mn,Outbound: Travel - US$ Mn
0,AFGHANISTAN,1995,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,AFGHANISTAN,1996,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,AFGHANISTAN,1997,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### Standardize Country Codes

In [42]:
uwto_pivoted['Country Code'] = uwto_pivoted['COUNTRY'].apply(lambda x: pycountry.countries.lookup(x).alpha_3)

In [43]:
uwto_pivoted.drop(columns=['COUNTRY'], inplace=True)

### Save the Data

In [44]:
uwto_pivoted.to_pickle("../data/interim/UWTO.pickle")